## 58스텝 : VGG 구현하기

![그림](./Posting3/VGG구조.png)

- 3*3 커널 크기, 출력층 64개       
- 풀링
- 3*3 커널 크기, 출력층 128개  
- 풀링
- 3*3 커널 크기, 출력층 256개     
- 풀링           
- 3*3 커널 크기, 출력층 512개         
- 풀링           
- 3*3 커널 크기, 출력층 512개      
- 풀링    
- 리니어 
- 드롭아웃        
- 리니어
- 드롭아웃      
- 리니어

> - 3*3 합성곱층 사용         
> - 풀링 하면 2배로 증가      
> - 완전 연결 계층에서는 드롭아웃 사용       
> - 활성화 함수로는 ReLU 사용

In [ ]:
import dezero.function as F
import dezero.layers as L

#모델 클래스를 상속받아 사용함
class VGG16(Model):
    def __init__(self):
        
        #부모 클래스의 이닛을 물려받아 사용
        super().__init__() 
        
        self.conv1_1 = L.Conv2d(64,kernel_size=3, stride=1, pad=1)
        self.conv1_2 = L.Conv2d(64,kernel_size=3, stride=1, pad=1)
        
        self.conv2_1 = L.Conv2d(128,kernel_size=3, stride=1, pad=1)
        self.conv2_2 = L.Conv2d(128,kernel_size=3, stride=1, pad=1)
        
        self.conv3_1 = L.Conv2d(256,kernel_size=3, stride=1, pad=1)
        self.conv3_2 = L.Conv2d(256,kernel_size=3, stride=1, pad=1)
        self.conv3_3 = L.Conv2d(256,kernel_size=3, stride=1, pad=1)
        
        self.conv4_1 = L.Conv2d(512,kernel_size=3, stride=1, pad=1)
        self.conv4_2 = L.Conv2d(512,kernel_size=3, stride=1, pad=1)
        self.conv4_3 = L.Conv2d(512,kernel_size=3, stride=1, pad=1)
        
        self.conv5_1 = L.Conv2d(512,kernel_size=3, stride=1, pad=1)
        self.conv5_2 = L.Conv2d(512,kernel_size=3, stride=1, pad=1)
        self.conv5_3 = L.Conv2d(512,kernel_size=3, stride=1, pad=1)
        
        self.fc6 = L.Linear(4096)
        self.fc7 = L.Linear(4096)
        self.fc8 = L.Linear(1000)
        
        #계층 연결
    def forward(self, x):
        
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = F.pooling(x, 2, 2)
        
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = F.pooling(x, 2, 2)
        
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = F.pooling(x, 2, 2)
        
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = F.pooling(x, 2, 2)
        
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = F.pooling(x, 2, 2)
        
        
        #합성곱층에서 완전연결층으로 변환하기 위한 reshape
        #합성곱층에서는 4차원 텐서를 처리하지만 완전연결계층에서는 2차원 텐서를 처리
        #완전연결계층에 데이터를 제공하기 전에 2차원 텐서로 변환
        
        x = F.reshape(x, (x.shpae[0], -1))
        x = F.dropout(F.relu(self.fc6(x)))
        x = F.dropout(F.relu(self.fc6(x)))
        x = self.fc8(x)
        
        return x

### 이미 학습된 가중치 데이터 가져오기

In [ ]:
class VGG16(Model):
    WEIGHTS_PATH = 'https://github.com/koki0702/dezero-models/releases/download/v0.1/vgg16.npz'

    #여기 pretrained=False
    def __init__(self, pretrained=False):
        # 이 내용은 위와 같음

        #만약 pretrained==True 이면,
        #위의 명시된 주소에서 학습된 가중치를 가져와 적용할 수 있도록 한다.
        if pretrained:
            weights_path = utils.get_file(VGG16.WEIGHTS_PATH)
            self.load_weights(weights_path)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = F.pooling(x, 2, 2)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = F.pooling(x, 2, 2)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = F.pooling(x, 2, 2)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = F.pooling(x, 2, 2)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = F.pooling(x, 2, 2)
        x = F.reshape(x, (x.shape[0], -1))
        x = F.dropout(F.relu(self.fc6(x)))
        x = F.dropout(F.relu(self.fc7(x)))
        x = self.fc8(x)
        return x

    @staticmethod
    def preprocess(image, size=(224, 224), dtype=np.float32):
        image = image.convert('RGB')
        if size:
            image = image.resize(size)
        image = np.asarray(image, dtype=dtype)
        image = image[:, :, ::-1]
        image -= np.array([103.939, 116.779, 123.68], dtype=dtype)
        image = image.transpose((2, 0, 1))
        return image

### 위의 pretrained를 사용하려면 이렇게

In [ ]:
import numpy as np
from dezero.models import VGG16

model = VGG16(pretrained=True)

x = np.random.randn(1,3,224,224).astype(np.float32)
model.plot(x)

### 계산그래프로 살펴보자

![계산그래프](./Posting3/VGG계산그래프.png)

## 3.학습된 VGG 구현하기

![샘플이미지](./Posting3/샘플이미지투입.png)

In [ ]:
    @staticmethod
    def preprocess(image, size=(224, 224), dtype=np.float32):
        image = image.convert('RGB')
        if size:
            image = image.resize(size)
        image = np.asarray(image, dtype=dtype)
        image = image[:, :, ::-1]
        
        #이미지넷으로 미리 구해둔 채널의 평균을 이미지에서 빼주는 처리
        image -= np.array([103.939, 116.779, 123.68], dtype=dtype)
        
        #이미지를 RGB -> BGR 순으로 재정렬
        #RGB(0,1,2) -> (2,1,0)
        image = image.transpose((2, 0, 1))
        return image

> - PIL.Image 형태의 데이터 타입을 dezero에서 다룰 수 있도록 ndarray 타입으로 변경 필요        
> - 이 함수를 VGG내부의 preprocess에 적용해두었음.        
> - preprocess 함수를 통해 (224,224)크기의 ndarray 타입으로 변환됨을 알 수 있음             
> - 또, 미지의 데이터셋을 학습된 가중치에 투입할 수 있도록 VGG에서 사용된 전처리 과정을 정확히 거쳐가도록 처리함(위의 주석 부분)

### 불러온 이미지 투입시켜보기

In [ ]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from PIL import Image
import dezero
from dezero.models import VGG16


url = 'https://github.com/oreilly-japan/deep-learning-from-scratch-3/raw/images/zebra.jpg'
img_path = dezero.utils.get_file(url)
img = Image.open(img_path)

x = VGG16.preprocess(img)

################ -------------- 여기까지는 동일
#배치용 축 추가 
#기존의 전처리 마친 데이터는 (3,224,224) -> (1, 3, 224,224)
x = x[np.newaxis]

model = VGG16(pretrained=True)
with dezero.test_mode():
    y = model(x)
predict_id = np.argmax(y.data)

model.plot(x, to_file='vgg.pdf')
labels = dezero.datasets.ImageNet.labels()
print(labels[predict_id])

# Step 59 : RNN을 활용한 시계열 데이터 처리

- 지금까지의 신경망 : feed-forward(피드포워드)      
- 데이터 구조가 한 방향으로, 순방향으로만 데이터를 입력하는 구조        
- 순환 신경망

![순환신경망 구조](./Posting3/순환신경망.png)

### RNN계층 구현

- 이번 시각의 입력 데이터(시계열 데이터)인 $Xt$ 로,             
- 은닉 상태 $ht$를 출력하는 RNN

![rnn식](./Posting3/rnn식.png)

In [ ]:
class RNN(Layer):
    def __init__(self, hidden_size, in_size=None):
        super().__init__()
        
        self.x2h = Linear(hidden_size, in_size=in_size)
        self.h2h = Linear(hidden_size, in_size=in_size, nobias=True)
        self.h = None
        
    def reset_state(self):
        self.h = None
        
    def forward(self, x):
        
        #은닉 상태가 하나도 정해지지 않았다면
        #첫투입이므로 x만 이용하여 새로운 은닉벡터 계산
        if self.h is None:
            h_new = F.tanh(self.x2h(x))
            
        else:
            #위의 rnn식을 통해 이번 층의 입력을 바탕으로 한 새로운 은닉 상태 추출
            h_new = F.tanh(self.x2h(x)+ self.h2h(self.h))
            self.h = h_new
            return h_new

In [ ]:
class Linear(Layer):
    def __init__(self, out_size, nobias=False, dtype=np.float32, in_size=None):
        super().__init__()
        self.in_size = in_size
        self.out_size = out_size
        self.dtype = dtype

        self.W = Parameter(None, name='W')
        if self.in_size is not None:
            self._init_W()

        if nobias:
            self.b = None
        else:
            self.b = Parameter(np.zeros(out_size, dtype=dtype), name='b')

    def _init_W(self, xp=np):
        I, O = self.in_size, self.out_size
        W_data = xp.random.randn(I, O).astype(self.dtype) * np.sqrt(1 / I)
        self.W.data = W_data

    def forward(self, x):
        if self.W.data is None:
            self.in_size = x.shape[1]
            xp = cuda.get_array_module(x)
            self._init_W(xp)

        y = F.linear(x, self.W, self.b)
        return y

### RNN계층에 데이터 주기

In [ ]:
import numpy as np
import dezero.layers as L

#은닉층 크기만 전달하고, 인풋데이터는 들어오는 사이즈에 따라 그대로 수용하도록 함
rnn = L.RNN(10)

x = np.random.rand(1,1)
h = rnn(x)

y = rnn(np.random.rand(1,1))

![rnn1층](./Posting3/rnn1층.png)        

![rnn2층](./Posting3/rnn2층.png)

### RNN 계층 구현하기

In [ ]:
from dezero import Model
import dezero.functions as F
import dezero.layers as L

class SimpleRNN(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        #RNN돌리고
        self.rnn = L.RNN(hidden_size)
        
        #최종출력을 내놓음
        self.fc = L.Linear(out_size)
        
    def reset_state(self):
        self.rnn.reset_state()
        
    def forward(self, x):
        h = self.rnn(x)
        y = self.fc(h)
        
        return y

### 심플 RNN모델 학습시켜보기

In [ ]:
#1,1 사이즈에 맞게 random을 만들어줌
seq_data = [np.random.randn(1,1) for _ in range(1000)]

xs = seq_data[0:1]
ts = seq_data[1:]

model = SimpleRNN(10,1)

loss, cnt = 0,0

for x, t in zip(xs, ts):
    y = model(x)
    loss += F.mean_squared_error(y, t)
    
    cnt += 1
    
    #Truncated BPTT, 2개째에서 끊어줌
    #이전의 은닉 상태를 유지해야 한다는 점이 중요!
    if cnt == 2:
        model.cleargrads()
        loss.backward()
        break

![손실함수이후의그래프](./Posting3/전체모습.png)

> 역전파에서 끊어주되      
> RNN의 은닉 상태가 유지된다는 것

![트룬케이트](./Posting3/truncated.png)

### 연결을 끊어주는 메서드

In [ ]:
class Variable:
    
    def unchain(self):
        self.creator = None
        
    #마주치는 모든 변수들의 creator를 none으로    
    def unchained_backward(self):
        
        #아직 끊어지지 않은 연결에 대해서는 끊어내기 위해
        if self.creator is not None:
            funcs = [self.creator]
            
            while funcs:
                
                #가장 최근의 함수를 꺼내서
                f = funcs.pop()
                
                #그 변수를 꺼내서
                for x in f.inputs:
                    
                    #그 인풋의 체인을 끊어준다.
                    if x.creator is not None:
                        funcs.append(x.creator)
                        x.unchain()

- 기존에 변수-함수-변수를 연결해주던 creator 를 None으로 설정하여 끊어주는 역할

### 사인파 예측(RNN테스트)        

- 시퀀셜한 사인그래프를 예측해보자.

In [ ]:
class SinCurve(Dataset):

    def prepare(self):
        num_data = 1000
        dtype = np.float64

        x = np.linspace(0, 2 * np.pi, num_data)
        noise_range = (-0.05, 0.05)
        noise = np.random.uniform(noise_range[0], noise_range[1], size=x.shape)
        if self.train:
            y = np.sin(x) + noise
        else:
            y = np.cos(x)
        y = y.astype(dtype)
        self.data = y[:-1][:, np.newaxis]
        self.label = y[1:][:, np.newaxis]

        
import numpy as np
import dezero
import matplotlib.pyplot as plt

train_set = dezero.datasets.SinCurve(train=True)

print(len(train_set))
print(train_set[0])
print(train_set[1])
print(train_set[2])

xs = [example[0] for example in train_set]

#1이어야 하지 않나?
ts = [example[1] for example in train_set]

plt.plot(np.arrange(len(xs)), xs, label='xs')
plt.plot(np.arrange(len(ts)), ts, label='ts')
plt.show()

![노이즈가있는](./Posting3/노이즈가있는.png)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dezero
from dezero import Model
import dezero.functions as F
import dezero.layers as L

#하이퍼 파라미터 설정
max_epoch = 100
hidden_size = 100

#bptt길이
bptt_length = 30

train_set = dezero.datasets.SinCurve(train=True)

#시퀀셜 데이터 길이 == 트레인 셋 길이
seqlen = len(train_set)


class SimpleRNN(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.rnn = L.RNN(hidden_size)
        self.fc = L.Linear(out_size)

    def reset_state(self):
        self.rnn.reset_state()

    def __call__(self, x):
        h = self.rnn(x)
        y = self.fc(h)
        return y


model = SimpleRNN(hidden_size, 1)
optimizer = dezero.optimizers.Adam().setup(model)

#트레이닝 시작
for epoch in range(max_epoch):
    
    #처음의 상태및 loss, count(bptt세기 위한 변수)를 0로 초기화
    model.reset_state()
    loss, count = 0, 0

    
    for x, t in train_set:
        
        #학습을 위해 모델에 투입될 x의 형태를 리셰이프
        #dezero에서는 입력 데이터가 2차원/4차원 텐서만 가능
        x = x.reshape(1, 1)
        y = model(x)
        loss += F.mean_squared_error(y, t)
        count += 1

        #에포크가 미리 지정해둔 bptt의 길이와 일치하거나, 모든 데이터셋 학습이 끊나면 unchain()
        if count % bptt_length == 0 or count == seqlen:
            model.cleargrads()
            loss.backward()
            loss.unchain_backward()
            optimizer.update()

    avg_loss = float(loss.data) / count
    print('| epoch %d | loss %f' % (epoch + 1, avg_loss))

### 테스트를 위해 코사인 예측시켜보기

In [ ]:
# Plot

#리얼 코사인
#np.lineapce(start, stop, num)
xs = np.cos(np.linspace(0, 4 * np.pi, 1000))
model.reset_state()
pred_list = []

with dezero.no_grad():
    for x in xs:
        x = np.array(x).reshape(1, 1)
        y = model(x)
        pred_list.append(float(y.data))

plt.plot(np.arange(len(xs)), xs, label='y=cos(x)')
plt.plot(np.arange(len(xs)), pred_list, label='predict')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## step 60 : LSTM과 데이터 로더

__두가지 개선 포인트__         

##### 1. 시계열 데이터용 '데이터 로더' 만들기        
- 모델에 데이터를 1개씩 전달하여(for x in range(train_data):) 이용하였는데, 이를 개선     
- 시계열 데이터용 데이터 로더를 만들어, 미니배치 단위로 데이터를 전달할 수 있도록 구현 개선     

##### 2. RNN 대신 LSTM 계층을 이용함         
- 더 나은 인식 성능



###### 이후 사인파 학습을 다시 한 번 시도해보자!

### 시계열 데이터용 데이터 로더 만들기     

- 미니배치 단위로 처리하려면, 커다란 하나의 데이터셋을 미니배치 단위로 다르게 지정해야 한다.

In [ ]:
class SeqDataLoader(DataLoader):
    def __init__(self, dataset, batch_size, gpu=False):
        super()__init__(dataset=dataset, batch_size=batch_size, shuffle=False, gpu=gpu)
        
    def __next__(self):
        
        #만약 최대 수가 되면 멈추기!
        if self.iteration >= self.max_iter:
            self.reset()
            raise StopIteration
            
        #나누어 몫을 jump로 기억
        jump = self.data_size//self.batch_size
        
        #배치 인덱스 = [새로 시작하는 인덱스]
        #이번 점프 + 나머지(self.iteration % self.data_size)
        batch_index = [(i*jump) + self.iteration % self.data_size for i in range(self.batch_size)] 
        
        batch = [self.dataset[i] for i in batch_index]
        
        xp = cuda.cupy if self.gpu else np
        x = [example[0] for example in batch]
        t = [example[1] for example in batch]
        
        self.iteration += 1
        
        return x, t

### 적용해보기

In [ ]:
train_set = dezero.datasets.SinCurve(train=True)
dataloader = SeqDataLoader(train_set, batch_size=3)

x, t = next(dataloader)

## LSTM 계층 구현

![LSTM](./Posting3/LSTM1.png)

![LSTM](./Posting3/LSTM2.png)

![LSTM](./Posting3/LSTM3.png)

In [ ]:
class LSTM(Layer):
    def __init__(self, hidden_size, in_size=None):
        super().__init__()

        H, I = hidden_size, in_size
        self.x2f = Linear(H, in_size=I)
        self.x2i = Linear(H, in_size=I)
        self.x2o = Linear(H, in_size=I)
        self.x2u = Linear(H, in_size=I)
        self.h2f = Linear(H, in_size=H, nobias=True)
        self.h2i = Linear(H, in_size=H, nobias=True)
        self.h2o = Linear(H, in_size=H, nobias=True)
        self.h2u = Linear(H, in_size=H, nobias=True)
        self.reset_state()

        #기억셀 c, 은닉벡터 h
    def reset_state(self):
        self.h = None
        self.c = None

    def forward(self, x):
        #첫 시도
        if self.h is None:
            f = F.sigmoid(self.x2f(x))
            i = F.sigmoid(self.x2i(x))
            o = F.sigmoid(self.x2o(x))
            u = F.tanh(self.x2u(x))
        else:
            f = F.sigmoid(self.x2f(x) + self.h2f(self.h))
            i = F.sigmoid(self.x2i(x) + self.h2i(self.h))
            o = F.sigmoid(self.x2o(x) + self.h2o(self.h))
            u = F.tanh(self.x2u(x) + self.h2u(self.h))

            #기억셀이 없을 경우(첫 트라이)
        if self.c is None:
            c_new = (i * u)
        else:
            c_new = (f * self.c) + (i * u)

        h_new = o * F.tanh(c_new)

        self.h, self.c = h_new, c_new
        return h_new

### LSTM로 sin학습하기

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dezero
from dezero import Model
from dezero import SeqDataLoader
import dezero.functions as F
import dezero.layers as L


max_epoch = 100
batch_size = 30
hidden_size = 100
bptt_length = 30

#==============================================================
#개선 1:시계열 데이터 로더 적용
train_set = dezero.datasets.SinCurve(train=True)
dataloader = SeqDataLoader(train_set, batch_size=batch_size)
seqlen = len(train_set)


class BetterRNN(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        
        #==================================
        #개선 2: LSTM층 사용
        self.rnn = L.LSTM(hidden_size)
        self.fc = L.Linear(out_size)

    def reset_state(self):
        self.rnn.reset_state()

    def __call__(self, x):
        y = self.rnn(x)
        y = self.fc(y)
        return y

model = BetterRNN(hidden_size, 1)
optimizer = dezero.optimizers.Adam().setup(model)

for epoch in range(max_epoch):
    model.reset_state()
    loss, count = 0, 0

    for x, t in dataloader:
        y = model(x)
        loss += F.mean_squared_error(y, t)
        count += 1

        if count % bptt_length == 0 or count == seqlen:
            model.cleargrads()
            loss.backward()
            loss.unchain_backward()
            optimizer.update()
    avg_loss = float(loss.data) / count
    print('| epoch %d | loss %f' % (epoch + 1, avg_loss))

# Plot
xs = np.cos(np.linspace(0, 4 * np.pi, 1000))
model.reset_state()
pred_list = []

with dezero.no_grad():
    for x in xs:
        x = np.array(x).reshape(1, 1)
        y = model(x)
        pred_list.append(float(y.data))

plt.plot(np.arange(len(xs)), xs, label='y=cos(x)')
plt.plot(np.arange(len(xs)), pred_list, label='predict')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## 칼럼

#### 1. 함수와 계층 추가             

- 텐서 곱을 해주는 tensorDot         
- 배치 정규화를 해주는 batchNorm    


#### 2. 메모리 사용 효율 개선          

- 딥러닝 프레임워크는 메모리의 효율적인 사용량도 중요한 토픽     
- esp, 대규모 신경망은 메모리를 많이 사용하기 때문에 데이터를 물리적인 메모리에 다 담지 못하는 문제가 흔히 발생함

- 현재는 순전파시의 계산 결과를 모두 기억해놓도록 설계했지만
- tanh의 경우, 순전파가 없이도 역전파를 계산 가능하므로($1-y^2$) 이를 취사선택할 수 있도록 확장하는 등

- [Aggressive Buffer Release](https://docs.google.com/document/d/1CxNS2xg2bLT9LoUe6rPSMuIuqt8Gbkz156LTPJPT3BE/edit#)   

- Abstract
This is a proposal to reduce the memory consumption of the computational graph built for backprop. The reduction is achieved by dropping arrays that are not needed for the backward method of each function node. It requires restructuring the graph definition, and thus it requires a slight modification to the API. 

(이 제안은 백프로파게이션에서의 메모리 소비를 감소시키도록 하는 제안이다. 이 reduction은 역전파 시 필요하지 않은 array를 제외(drop)함으로써 이루어진다. 해당 연산을 위해서는 그래프 정의를 재구축해야 하며, API에 다소의 변경이 필요하다.) 

### 3. 정적 계산 그래프와 ONNX 지원     

- 현재 define-by-Run(동적) 방식으로 고안되어 있으므로, 이를 Define-and-Run(정적) 방식으로도 확대     
- ONNX 라는 데이터 포맷을 지원

[ONNX깃허브](https://github.com/onnx/onnx)

- Open Neural Network Exchange (ONNX) is an open ecosystem that empowers AI developers to choose the right tools as their project evolves. ONNX provides an open source format for AI models, both deep learning and traditional ML. It defines an extensible computation graph model, as well as definitions of built-in operators and standard data types. Currently we focus on the capabilities needed for inferencing (scoring).
(ONNX는 AI모델, deep learning 및 traditional ml모델을 위한 오픈소스 포맷을 제공한다. ONNX는 computational graph, 빌트-인 오퍼레이터 및 표준 데이터 타입을 확장 가능하도록 정의한다.)

- ONNX를 사용하면 기학습된 모델을 다른 프레임워크로 쉽게 이식할 수 있다.

ONNX is widely supported and can be found in many frameworks, tools, and hardware. Enabling interoperability between different frameworks and streamlining the path from research to production helps increase the speed of innovation in the AI community. We invite the community to join us and further evolve ONNX.


### 4. PyPI 에 공개     
- 파이썬 패키지로써 PyPI에 공개     

### 5. API 문서 준비        
- 사용자를 위한 가이드 문서 준비       
- docstring    
- Sphinx를 이용하여 PDF, HTML 형태로 API 문서 뽑아내기 [사용법](https://tech.ssut.me/start-python-documentation-using-sphinx/)

#### 그외, 로고 준비 및 구현 예 추가(+GAN, VAE, Style Transfer 등..)

## C.구글 코랩에서 실행

[이 링크로 들어간다면!](https://colab.research.google.com/github/WegraLee/deep-learning-from-scratch-3/blob/master/examples/mnist_colab_gpu.ipynb)